# Import Libraries

In [14]:
!pip install -r requirements.txt -q

/Users/hedegaard/Documents/M3-TWITTERPROJECT/DDBMS-Submission/.conda/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=56494) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:

# Datahandling
import requests
import os
import re
import numpy as np
import pandas as pd
import seaborn as sns
import tqdm

#Setfit
from setfit import SetFitModel

## Data Import

In [25]:
# Data import 
data_test = pd.read_csv('output/tesla.csv')

In [26]:
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www.\S+', '', text)
    # Remove mentions (@username)
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags (but keep the text)
    text = re.sub(r'#', '', text)
    # Remove emojis and special characters
    text = re.sub(r'[^\w\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    return text

# Data Cleaning
data_test['Tweet_text'] = data_test['Tweet_text'].apply(clean_text)  # Custom cleaning function
data_test.dropna(subset=['Tweet_text'], inplace=True)

## SetFit Model Predictions

In [27]:
# Indlæs den gemte model
loaded_model = SetFitModel.from_pretrained("setfit_model")


# Antag at data_test er et pandas DataFrame med en 'Tweet_text' kolonne
# Forbered data til prediction
predictions = []


# Gennemgå hver tweet i data_test
for tweet in data_test['Tweet_text']:
    # Lav prediction på den enkelte tweet
    prediction = loaded_model([tweet])[0]
    # Konverter tensor til integer
    predictions.append(int(prediction))


# Tilføj predictions til data_test DataFrame
data_test['Label'] = predictions


# Valgfrit: Udskriv nogle eksempler
print("\nEksempler på predictions:")
print(data_test[['Tweet_text', 'Label']].head())


# Valgfrit: Gem predictions til CSV
data_test.to_csv('tesla_predictions_output.csv', index=False)


Eksempler på predictions:
                                          Tweet_text  Label
0  made it from la to new york city in my cybertr...      1
1  took our nephew for an fsd ride in a cybertruc...      0
2  ota update 2024395 for  is installing with fsd...      0
3  the ultimate test for cybertruck fsd\n\ntimes ...      0
4  with cybertruck fsd now released it now makes ...      1


In [13]:
data_test

,Tweet_id,User_name,Tweet_text,Language,Hashtags,Created_at,Search_Keyword,Predicted_Label,Prediction_Probability,Label
0,1853729719239233937,Elon Musk (Parody),would you buy a tesla phone\n\ncheaper and bet...,en,[],2024-11-05 09:23:00+00:00,Tesla,0,0.997223,0
1,1862546527144603906,Ken Synopsis & Vancouver Real Estate,optimus is going to be wild,en,[],2024-11-29 17:17:51+00:00,Tesla,0,0.997368,0
2,1862169330286280855,Elon Musk (Parody),should tesla make a phone\n\nbetter than iphon...,en,[],2024-11-28 16:19:00+00:00,Tesla,1,0.997845,1
3,1862166333234012621,Tesla,happy thanksgiving from our family to yours,en,[],2024-11-28 16:07:05+00:00,Tesla,1,0.997964,1
4,1862226046277337596,Elon Musk - Parody,hello friends\n\nwould you accept this tesla m...,en,[],2024-11-28 20:04:22+00:00,Tesla,0,0.997258,0
...,...,...,...,...,...,...,...,...,...,...
182,1860052999185531279,Canadian Jennifer 🇨🇦,would you have liked the original mock up cybe...,en,[],2024-11-22 20:09:27+00:00,CyberTruck,0,0.997320,0
183,1860028251273593198,Greggertruck,bummer fsd doesnt work with the tailgate down...,en,[],2024-11-22 18:31:07+00:00,CyberTruck,1,0.997655,1
184,1862669600690717157,Stunt Dummy,here we go again this tesla cybertruck owner j...,en,['CyberTruck'],2024-11-30 01:26:54+00:00,CyberTruck,0,0.997267,0
185,1861838571482784017,Jeff 💙✌️,does anyone know what the target button is for...,en,[],2024-11-27 18:24:41+00:00,CyberTruck,0,0.997363,0
